In [2]:
## Dependencies

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from area import area
import geopy
from geopy import Point, distance
from statistics import mean, median

# ETL Notebook
## New notebook to continue to streamline ETL of baseball field data

### 12/29/22
# idea
create a new id column. copy the school id for the high schools and assign ids to the fields that dont have them. use these column to match wo sheets with parameters, icon info, ect, college enrollment, seating capacity 

Working well. outputs a single csv with all relevent area and distance info including mean median max min calculations

matches high school names to school id numbers from mhsaa source to pull division and enrollment numbers
        

In [2]:
#### Load data from kml file exported by Google Earth

file_path = ('data/kml/ballparks.kml')

with open(file_path) as file:

    xml_data = file.read()



# Initialize soup variables for parsing file
soup = BeautifulSoup(xml_data, 'xml')

folders = soup.Document.Folder
list = soup.Document.Folder.find_all('Folder')
# layers = soup.Document.Folder.Folder
# polygons = soup.Document.Folder.Placemark.Polygon

In [6]:
## Create a dataframe to hold the data parsed from xml
df = pd.DataFrame(columns=['field', 'foul', 'fop'])

failed = []

## Loop through the folders and extract the data
i = 0   

for i in range(len(list)):
    try:
        folders = list[i]
        field_name = folders.find('name').text
        foul = folders.find_all('coordinates')[0].text
        fop = folders.find_all('coordinates')[1].text

        row = {
            'field': field_name,
            'foul': foul,
            'fop': fop
        }

        i+=1
    except: 
        ## add name of folder to a list of failed folders

        failed.append(list[i].find('name').text)
        i+=1
        continue

        


    df = df.append(row, ignore_index=True)

C:\Users\Justin\AppData\Local\Temp\ipykernel_15968\2598616467.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_15968\2598616467.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_15968\2598616467.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_15968\2598616467.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Loca

In [7]:
print(failed)

['Union City Youth Field - youth']


In [8]:

## Cleaning
# remove new line and and space characters from coordinates
df = df.replace(r'\n','', regex=True) 
df = df.replace(r'\t','', regex=True) 

## Drop any duplicate rows
df = df.drop_duplicates(subset=['field'], keep='first')

## Drop any rows with empty fields
df = df[(df != 0).all(1)]

In [9]:
## Parse field names to get level column using regex
import re
re_mlb = re.compile(r'mlb', re.IGNORECASE)
re_pro = re.compile(r'pro', re.IGNORECASE)
re_college = re.compile(r'college', re.IGNORECASE)
re_youth = re.compile(r'youth', re.IGNORECASE)
re_muni = re.compile(r'muni', re.IGNORECASE)

df['level'] = df['field'].apply(lambda x: 'mlb' 
        if re_mlb.search(x) else 'pro' 
        if re_pro.search(x) else 'college' 
        if re_college.search(x) else 'youth' 
        if re_youth.search(x) else 'muni' 
        if re_muni.search(x) else 'high_school')

#### SIZE FACTOR TO LATER USE IN VISUALIZATIONS
# mlb = 5000
# pro = 4000
# college = 3000
# youth = 300
# muni = 2000
# high_school = 1000

df['size_factor'] = df['level'].apply(lambda x: 5000
        if x == 'mlb' else 3000
        if x == 'pro' else 2000
        if x == 'college' else 100
        if x == 'youth' else 1000
        if x == 'muni' else 300)



# clean up the field names
# remove the level from the field name
df['field'] = df['field'].str.replace(r'MLB', '')
df['field'] = df['field'].str.replace(r'pro', '')
df['field'] = df['field'].str.replace(r'college', '')
df['field'] = df['field'].str.replace(r'High School', 'HS')
# remove - from end of field name
df['field'] = df['field'].str.replace(r'- $', '')

## Output test csv
# df.to_csv('TEMP/level2_tost.csv', index=False)

C:\Users\Justin\AppData\Local\Temp\ipykernel_15968\2811428785.py:40: FutureWarning: The default value of regex will change from True to False in a future version.
  df['field'] = df['field'].str.replace(r'- $', '')


In [10]:
## Transform DATA
# taken from clean_notebook_parse

## Create a (lat, long) pair for home plate 
df['home_plate'] = df['foul'].str.split(' ').str[0]

## Apply lambda function to format polygon coordinates and pass to the area function
# #OPutput in square meters
df['foul_area'] = df['foul'].apply(lambda x: area({'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]}))
df['fop_area'] = df['fop'].apply(lambda x: area({'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]}))

# Convert the area to square feet
df['foul_ft'] = df['foul_area'].apply(lambda x: x*10.7639)
df['fop_ft'] = df['fop_area'].apply(lambda x: x*10.7639)
# round the area to an integer
df['foul_ft'] = df['foul_ft'].round(0)
df['fop_ft'] = df['fop_ft'].round(0)

# drop the square meter area columns
df = df.drop(['foul_area', 'fop_area'], axis=1)

########12/29/22 #########
### Need to hang on to the polygon coordinates for the distance calculations
# # drop the polygon coordinates
# df = df.drop(['foul', 'fop'], axis=1)
############################

# calculate the ratio of foul ground to total area and convert to percentage
df['foul_pct'] = (df['foul_ft']/(df['fop_ft']+df['foul_ft']))*100
# round the percentage to 2 decimal places
df['foul_pct'] = df['foul_pct'].round(2)

### Split the home_plate coordinates into lat and long columns
df['h_lon'] = df['home_plate'].str.split(',').str[0]
df['h_lat'] = df['home_plate'].str.split(',').str[1]

## Drop the home_plate column
df = df.drop(['home_plate'], axis=1)

# # Outputs a clean CSV with area and home plate coordinates
# df.to_csv('data/viz/area_table.csv', index=False)





In [11]:
### Adding the code to calculate the distance in the same notebook

## Code taken from distance_notebook.ipynb on December 29th for testing


### Extract the home_plate coordinates by takeing the first coordinate in the foul coordinates column
df['home_plate'] = df['foul'].str.split(' ').str[0]

## drop the last 2 characters from the home_plate coordinates
df['home_plate'] = df['home_plate'].str[:-2]

### drop every ',0' from the fop coordinates
df['fop'] = df['fop'].str.replace(',0', '')

### reverse the order of the coordinates in the home_plate column
df['home_plate'] = df['home_plate'].str.split(',').str[::-1].str.join(',')

### Creaty a list of tuples for the fop coordinates
df['fop_list'] = df['fop'].apply(lambda x: [tuple(map(float, coord.split(','))) for coord in x.split()])

## Reverse the order of the values each coordinate tuple
df['fop_list'] = df['fop_list'].apply(lambda x: [tuple(reversed(coord)) for coord in x])

### use the geopy distance method to get the distance between home_plate and each coordinate in the fop_list column for each row of the dataframe


### calculate the distance between home_plate and each coordinate in the fop_list column
df['distance'] = df.apply(lambda x: [distance.distance(x['home_plate'], coord).ft for coord in x['fop_list']], axis=1)

df['distance']

### Split the dataframe based on if the level is youth or not

youth_df = df[df['level'] == 'youth']
upper_df = df[df['level'] != 'youth']

### Drop any distance values from the upper_df less than 200 feet and calculate the min and max distance
upper_df['distance'] = upper_df['distance'].apply(lambda x: [i for i in x if i > 180])
upper_df['min_distance'] = upper_df['distance'].apply(lambda x: min(x))
upper_df['max_distance'] = upper_df['distance'].apply(lambda x: max(x))
upper_df['avg_distance'] = upper_df['distance'].apply(lambda x: mean(x))
upper_df['median_distance'] = upper_df['distance'].apply(lambda x: median(x))

## Drop any zero values from the youth_df and calculate the min and max distance

youth_df['distance'] = youth_df['distance'].apply(lambda x: [i for i in x if i > 100])
youth_df['min_distance'] = youth_df['distance'].apply(lambda x: min(x))
youth_df['max_distance'] = youth_df['distance'].apply(lambda x: max(x))
youth_df['avg_distance'] = youth_df['distance'].apply(lambda x: mean(x))
youth_df['median_distance'] = youth_df['distance'].apply(lambda x: median(x))

# Recombine the dataframes
df = pd.concat([upper_df, youth_df])

## Round all the distance values to the nearest foot
df['min_distance'] = df['min_distance'].round(0)
df['max_distance'] = df['max_distance'].round(0)
df['avg_distance'] = df['avg_distance'].round(0)
df['median_distance'] = df['median_distance'].round(0)

### output the dataframe to a csv file
## Temp file for checking
# df.to_csv('TEMP/viz_book_distance.csv', index=False)




C:\Users\Justin\AppData\Local\Temp\ipykernel_15968\3594921718.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upper_df['distance'] = upper_df['distance'].apply(lambda x: [i for i in x if i > 180])
C:\Users\Justin\AppData\Local\Temp\ipykernel_15968\3594921718.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upper_df['min_distance'] = upper_df['distance'].apply(lambda x: min(x))
C:\Users\Justin\AppData\Local\Temp\ipykernel_15968\3594921718.py:40: SettingWithCopyWarning: 
A value is trying to be set o

In [3]:
# ## Merge the enrollment data with the dataframe


### Read the enrollment table from MHSAA website - 2022 enrollment
mhsaa_df = pd.read_csv('data/school_info/mhsaa_enrolment_2022.csv')

## Take area_df from above
area_df = df





### Import area df from a csv file (used for testing purposes)
# ### Open area data and merge with enrollment data
# area_df = pd.read_csv('data/viz/area_table.csv')

# area_df.head()

# area_df['level'].value_counts()



# ## export the mhsaa_df to csv
# mhsaa_df.to_csv('TEMP/mhsaa_df.csv', index=False)




NameError: name 'df' is not defined

In [5]:
### TEMP BLOCK TO GET LIST

# ## Create a list of the schools in the mhssa_df
school_list = mhsaa_df['school_name'].tolist()

print(school_list)

['Macomb - Dakota HS', 'Grand Blanc HS', 'East Kentwood HS', 'Dearborn - Fordson HS', 'Shelby Township - Utica Eisenhower HS', 'Northville HS', 'Rockford HS', 'Detroit - Cass Technical HS', 'Sterling Heights - Utica Stevenson HS', 'Clinton Township - Chippewa Valley HS', 'Howell HS', 'Lake Orion HS', 'Oxford HS', 'Troy HS', 'Clarkston HS', 'Ann Arbor - Pioneer HS', 'Canton - Salem HS', 'Canton - Plymouth HS', 'Canton HS', 'Holland - West Ottawa HS', 'Dearborn HS', 'West Bloomfield HS', 'Brighton HS', 'Novi HS', 'Hudsonville HS', 'Grand Haven HS', 'Hartland HS', 'Grand Rapids - Union HS', 'Southfield HS for the Arts & Technology', 'Fair Haven - Anchor Bay HS', 'Novi - Detroit Catholic Central HS', 'Ann Arbor - Huron HS', 'Bloomfield Hills HS', 'Monroe HS', "Macomb - L'Anse Creuse North HS", 'Utica HS', 'Detroit - Western International HS', 'Davison HS', 'Troy - Athens HS', 'Sterling Heights - Utica Ford II HS', 'Grandville HS', 'Holt HS', 'Saline HS', 'Lapeer HS', 'Belleville HS', 'Kala

In [13]:
# select just the high school level
hs_df = area_df[area_df['level'] == 'high_school']
other_df = area_df[area_df['level'] != 'high_school']

# hs_df.info()
# mhsaa_df.info()

In [14]:
import random

### Create a new column in the other_df called id

other_df['id'] = 0

## create unique IDs for each row in other_df
## Create alphanumeric id for each row
# the ID should be the first 2 letters of the field name and three digits
other_df['id'] = other_df['field'].apply(lambda x: x[:2] + str(random.randint(100, 199)))


C:\Users\Justin\AppData\Local\Temp\ipykernel_15968\2950605514.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_df['id'] = 0
C:\Users\Justin\AppData\Local\Temp\ipykernel_15968\2950605514.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_df['id'] = other_df['field'].apply(lambda x: x[:2] + str(random.randint(100, 199)))


In [15]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

hs_names = hs_df['field'].tolist()
enroll_names = mhsaa_df['school_name'].tolist()
# id_list = mhsaa_df['school_id'].tolist()

# set the treshold for the fuzzy match
treshold = 90
mat1 = []


## do the fuzzy match
for i in hs_names:
    mat1.append(process.extract(i, enroll_names, limit=1, scorer=fuzz.token_set_ratio))

hs_df['match'] = mat1


### Take the first item in the tuple
hs_df['match'] = hs_df['match'].apply(lambda x: x[0][0])

#### Lookup the hs_df['match'] in the mhsaa_df['school_name'] and return the school_id
hs_df['school_id'] = hs_df['match'].apply(lambda x: mhsaa_df[mhsaa_df['school_name'] == x]['school_id'].values[0])

## copy the school_id column to the id column
hs_df['id'] = hs_df['school_id']


C:\Users\Justin\AppData\Local\Temp\ipykernel_15968\1633521730.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs_df['match'] = mat1
C:\Users\Justin\AppData\Local\Temp\ipykernel_15968\1633521730.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs_df['match'] = hs_df['match'].apply(lambda x: x[0][0])
C:\Users\Justin\AppData\Local\Temp\ipykernel_15968\1633521730.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [16]:
### 12/30/22 ###
### NEW BLOCK TO ADD THE ID COLUMN TO THE NON HS FIELD DATA SHEET ###

#Infoformation with field capacity, college school enrollment and various other information
info_file = 'data/viz/upper_level_info.csv'

## read it to a dataframe
info_df = pd.read_csv(info_file, encoding = 'latin-1')

# create a list of the field names from the dataframe

field_names = other_df['field'].tolist()

# create a list of the field names from the info file

info_field_names = info_df['field'].tolist()

# create a list of the unique ids from the viz file

unique_ids = other_df['id'].tolist()

## Use fuzzywuzzy to match the field names from the info file to the viz file

# set the treshold for the fuzzy match
treshold = 90
mat1 = []


# output a list of the school_id from the mhsaa_df based on the fuzzy match


## do the fuzzy match
for i in info_field_names:
    mat1.append(process.extract(i, field_names, limit=1, scorer=fuzz.token_set_ratio))

info_df['match'] = mat1
# mhsaa_df['match'] = mat1

### Take the first item in the tuple

info_df['match'] = info_df['match'].apply(lambda x: x[0][0])

#### Lookup the info_df['match'] in the mhsaa_df['school_name'] and return the id

info_df['id'] = info_df['match'].apply(lambda x: other_df[other_df['field'] == x]['id'].values[0])


info_df.head()

,Unnamed: 0,field,field2,icon_full,league,team_name,icon_pair,capacity,undergrad_pop,match,id,notes
0,0,Commerica Park - Tigers,NaN,DET,MLB,Detroit Tigers,DET,"41,083",NaN,Commerica Park - Tigers,Co141,NaN
1,1,Coors Field - Largest Park in the MLB,NaN,COL,MLB,Colorado Rockies,COL,"50,398",NaN,Coors Field - Largest Park in the - mlb,Co191,NaN
2,2,Fenway Park - Smallest Park in the MLB,NaN,BOS,MLB,Boston Red Sox,BOS,"37,731",NaN,Fenway Park - Smallest Park in the - mlb,Fe157,NaN
3,3,Battle Creek - Robert L Nichols Field - muni,2nd field at Bailey Park with MCCU Field,muni,NaN,NaN,NaN,6000,NaN,Robert L Nichols Field (Battle Creek) - muni,Ro126,NaN
4,4,Boyne City County Park Muni,NaN,muni,NaN,NaN,NaN,175,NaN,Boyne City County Park Muni,Bo178,NaN


In [17]:
### Add the non high schools back to the dataframe

df = hs_df.append(other_df, ignore_index=True)

## Drop unnessisary columns
df.drop(['foul', 'fop', 'fop_list', 'distance'], axis=1, inplace=True)

df.info()

df['level'].value_counts()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   field            646 non-null    object 
 1   level            646 non-null    object 
 2   size_factor      646 non-null    int64  
 3   foul_ft          646 non-null    float64
 4   fop_ft           646 non-null    float64
 5   foul_pct         646 non-null    float64
 6   h_lon            646 non-null    object 
 7   h_lat            646 non-null    object 
 8   home_plate       646 non-null    object 
 9   min_distance     646 non-null    float64
 10  max_distance     646 non-null    float64
 11  avg_distance     646 non-null    float64
 12  median_distance  646 non-null    float64
 13  match            490 non-null    object 
 14  school_id        490 non-null    float64
 15  id               646 non-null    object 
dtypes: float64(8), int64(1), object(7)
memory usage: 80.9+ KB


C:\Users\Justin\AppData\Local\Temp\ipykernel_15968\2681302149.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = hs_df.append(other_df, ignore_index=True)


high_school    490
youth           82
college         37
muni            16
mlb             14
pro              7
Name: level, dtype: int64

In [18]:
### Output to csv #####
### TEMP FILES FOR TESTING
# df.to_csv('TEMP/TOST_viz_book.csv', index=True)
# info_df.to_csv('TEMP/TOST_upper_level_info.csv', index=True)

# ###### DON'T TURN ON UNITL ALREADY TESTED ######
# # Output files to visualization directory
# # Main book with all the spacial data
df.to_csv('data/tables/area_distance.csv', index=True)

# # Info file for non high school fields linked by unique id to the main book
info_df.to_csv('data/tables/stadium_info.csv', index=True)